## Preparation

In [4]:
import pandas as pd
import numpy as np
import re

### Identify AI related topics by looking top 2000 project's topic

In [23]:
seed = {
    'neural': 1,
    'ml': 1,
    'learning': 1,
    'gpt': 1,
    'llm': 1,
    'nlp': 1,
    'assistant': 1,
    'cnn': 1,
    'rnn': 1,
    'gan': 1,
    'model': 1,
    'transformer': 1,
    'recognition': 1,
    'detection': 1,
    'ai': 1,
    'diffusion': 1,
    'yolo': 1,
    'classifier': 1,
    'classification': 1,
    'artificial': 1,
    'bert': 1,
    'torch': 1,
    'jax': 1,
    'tensor': 1,
    'seq2seq': 1,
    'agent': 1,
    'bot': 1,
    'gpu': 1,
    'img': 1,
    'image': 1,
    'film': 1,
    'face': 1,
    'music': 1,
    'caffe': 1,
    'mxnet': 1,
    'vector': 1,
    'scikit': 1,
    'deep': 1,
    'natural': 1,
    'ocr': 1,
    'cv': 1,
    'llama': 1,
    'bard': 1,
    'claude': 1,
    'gpu': 1,
    'tpu': 1,
    'parameters': 1,
    'regression': 1,
    'intelligent': 1,
    'intelligence': 1,
    'att': 1,
    'adaboost': 1,
    'age-prediction': 1,
    'alpaca': 1,
    'bayesian': 1,
    'bias': 1,
    'causal': 1,
    'chatglm': 1,
    'computer-vision': 1,
    'decision-tree': 1,
    'embeddings': 1,
    'factorization': 1,
    'game-theory': 1,
    'cuda': 1,
}

def load_main(file):
    df_main = pd.read_csv(
        file,
        parse_dates=[ "pushed_at", "created_at", "updated_at" ]
    )
    df_main.drop_duplicates(ignore_index=True, inplace=True)
    df_main = df_main.astype({'owner_type': 'category', 'default_branch': 'category', 'language': 'category', 'lang': 'category'})
    return df_main

def is_known(topic):
    for k in seed:
        if topic.find(k) >= 0:
            return True
    return False        


def _split_by_topics(row):
    topics = row['topics']
    row['topic'] = [m.group(1) for m in re.finditer(r"'([-\w]+)'", topics)]
    if len(row['topic']) == 0: row['topic'] = 'NOTOPIC'
    row['primary_topic'] = row['topic'][0]    
    return row

def _walk_dataframe(df, func):
    df = df.apply(func, axis=1)
    return df

def expand(df):
    df = _walk_dataframe(df, _split_by_topics)
    df = df.explode(['topic'])
    return df

df_commit = pd.read_csv("commits-sum.csv", parse_dates=["author_date"], dtype={"verified": "int8", "branch": "category"})
df_topics = df_commit.groupby(["full_name"], as_index=False).agg(contributors=pd.NamedAgg(column="author_name", aggfunc="nunique")).sort_values("contributors", ascending=False).head(5000)
df_main = load_main("main.csv")
df_topics = df_topics.merge(df_main, how="inner", left_on="full_name", right_on="full_name")
df_topics = expand(df_topics)

df_top = df_topics.groupby("topic", as_index=False).agg(cnt=pd.NamedAgg(column="full_name", aggfunc="count")).sort_values("cnt", ascending=False)

In [27]:
df_top.describe(percentiles=[.75, .90, .95, .97, .98, .99])

,cnt
count,11143.000000
mean,3.899578
std,38.588373
min,1.000000
50%,1.000000
75%,2.000000
90%,5.000000
95%,9.000000
97%,15.000000
98%,21.000000


In [30]:
df_top.head(250).to_csv("to-pick.csv", index=False)

In [ ]:
    
topics = list(df_topics['topic'].unique())
topics.sort()
j = 0
for i, topic in enumerate(topics):
    #if not is_known(topic):
    if is_known(topic):
        print(f"{j}-{i}: {topic}")
        j += 1